In [65]:
# Core Modules
import pandas as pd
import numpy as np

# Basic modules for data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [66]:
test_df = pd.read_csv('clean_test.csv')
train_df = pd.read_csv('clean_train.csv')

In [67]:
test_df.shape

(418, 11)

In [68]:
train_df.shape

(891, 12)

In [69]:
final_df = pd.concat([train_df,test_df],axis=0)

In [70]:
final_df.shape

(1309, 12)

In [71]:
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields],drop_first=True)
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final,df1],axis=1)
        i = i + 1

    df_final = pd.concat([final_df,df_final],axis = 1)
    return df_final

In [72]:
cat_attributes = final_df[['Sex', 'Cabin', 'Embarked']]

In [73]:
final_df = category_onehot_multcols(cat_attributes)

Sex
Cabin
Embarked


In [74]:
final_df.shape

(1309, 198)

In [75]:
final_df = final_df.loc[:,~final_df.columns.duplicated()]
final_df.shape

(1309, 198)

In [76]:
df_train = final_df.iloc[:891,:]
df_test = final_df.iloc[891:,:]

In [77]:
df_train.shape

(891, 198)

In [78]:
df_test.shape

(418, 198)

In [79]:
df_test.drop(['Survived'],axis=1,inplace=True)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [80]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Columns: 198 entries, PassengerId to S
dtypes: float64(3), int64(4), object(2), uint8(189)
memory usage: 234.1+ KB


In [81]:
df_train = df_train.drop(['Name', 'Ticket'],axis=1)

In [82]:
x_train = df_train.drop(['Survived'],axis=1)
y_train = df_train['Survived']

In [83]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [84]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [85]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [87]:
df_test.shape

(418, 197)

In [88]:
df_train.shape

(891, 196)

In [89]:
df_test = df_test.drop(['Name', 'Ticket'],axis=1)

In [90]:
model_pred = model.predict(df_test)

In [93]:
pred = pd.DataFrame(model_pred)
sub_df = pd.read_csv('../../Datasets/titanic/gender_submission.csv')
datasets = pd.concat([sub_df['PassengerId'],pred],axis=1)

In [94]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   0            418 non-null    int32
dtypes: int32(1), int64(1)
memory usage: 5.0 KB


In [99]:
datasets.columns = ['PassengerId', 'Survived']
datasets.to_csv('submission.csv',index=False)